In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

#### Loading the Acquisition Data

In [2]:
acquisition_data_path = "G:/Stevens_Lecture/2019_Fall/FE690_Machine_Learning_in_Finance/HW2/2010Q1/Acquisition_2010Q1.txt"

AcquisitionColumnNames = (
    "LOAN_ID", "ORIG_CHN", "Seller.Name", 
    "ORIG_RT", "ORIG_AMT", "ORIG_TRM", "ORIG_DTE",
    "FRST_DTE", "OLTV", "OCLTV", "NUM_BO", 
    "DTI", "CSCORE_B", "FTHB_FLG", "PURPOSE", 
    "PROP_TYP", "NUM_UNIT", "OCC_STAT", "STATE", "ZIP_3", 
    "MI_PCT", "Product.Type", "CSCORE_C", "MI_TYPE", 
    "RELOCATION_FLG"
)

acquisition_df = pd.read_csv(
    acquisition_data_path,
    names=AcquisitionColumnNames,
    header=None,
    sep="|"
)

In [3]:
acquisition_df.columns

Index(['LOAN_ID', 'ORIG_CHN', 'Seller.Name', 'ORIG_RT', 'ORIG_AMT', 'ORIG_TRM',
       'ORIG_DTE', 'FRST_DTE', 'OLTV', 'OCLTV', 'NUM_BO', 'DTI', 'CSCORE_B',
       'FTHB_FLG', 'PURPOSE', 'PROP_TYP', 'NUM_UNIT', 'OCC_STAT', 'STATE',
       'ZIP_3', 'MI_PCT', 'Product.Type', 'CSCORE_C', 'MI_TYPE',
       'RELOCATION_FLG'],
      dtype='object')

#### Loading the Performance Data

In [4]:
performance_data_path = "G:/Stevens_Lecture/2019_Fall/FE690_Machine_Learning_in_Finance/HW2/2010Q1/Performance_2010Q1.txt"

PerformanceColumnNames = (
    "LOAN_ID", "Monthly.Rpt.Prd", "Servicer.Name", 
    "LAST_RT", "LAST_UPB", "Loan.Age", "Months.To.Legal.Mat", 
    "Adj.Month.To.Mat", "Maturity.Date", "MSA", 
    "Delq.Status", "MOD_FLAG", "Zero.Bal.Code", 
    "ZB_DTE", "LPI_DTE", "FCC_DTE","DISP_DT", 
    "FCC_COST", "PP_COST", "AR_COST", "IE_COST", 
    "TAX_COST", "NS_PROCS","CE_PROCS", "RMW_PROCS", 
    "O_PROCS", "NON_INT_UPB", "PRIN_FORG_UPB_FHFA", 
    "REPCH_FLAG", "PRIN_FORG_UPB_OTH", "TRANSFER_FLG"
)

performance_df = pd.read_csv(
    performance_data_path,
    names=PerformanceColumnNames,
    header=None,
    sep="|",
)

C:\Users\gaura\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
performance_df.columns

Index(['LOAN_ID', 'Monthly.Rpt.Prd', 'Servicer.Name', 'LAST_RT', 'LAST_UPB',
       'Loan.Age', 'Months.To.Legal.Mat', 'Adj.Month.To.Mat', 'Maturity.Date',
       'MSA', 'Delq.Status', 'MOD_FLAG', 'Zero.Bal.Code', 'ZB_DTE', 'LPI_DTE',
       'FCC_DTE', 'DISP_DT', 'FCC_COST', 'PP_COST', 'AR_COST', 'IE_COST',
       'TAX_COST', 'NS_PROCS', 'CE_PROCS', 'RMW_PROCS', 'O_PROCS',
       'NON_INT_UPB', 'PRIN_FORG_UPB_FHFA', 'REPCH_FLAG', 'PRIN_FORG_UPB_OTH',
       'TRANSFER_FLG'],
      dtype='object')

In [6]:
print(f"There are {len(acquisition_df.index)} rows in the Acquisition Dataframe")
print(f"There are {len(performance_df.index)} rows in the Performance Dataframe")

There are 323174 rows in the Acquisition Dataframe
There are 18080692 rows in the Performance Dataframe


In [14]:
# using trimmed performance data 
performance_df_new = performance_df[0:1000000]

In [15]:
len(performance_df_new.index)

1000000

#### Getting Initial Forclosure Data

In [16]:
FCC_DTE = performance_df_new['FCC_DTE'].notna()

forclosed = performance_df_new[FCC_DTE]

FORCLOSED = set(forclosed['LOAN_ID'])

def mapper(row):
    return row['LOAN_ID'] in FORCLOSED

acquisition_df['FCC'] = acquisition_df.apply(mapper, axis=1)

In [17]:
print(f"There are {len(FORCLOSED)} foreclosures!")

There are 93 foreclosures!


#### Exploring Delinquency Status

In [18]:
DELQ = performance_df_new['Delq.Status']
delq = set(DELQ)
print(delq)

{nan, '5', '35', '29', '49', '52', '63', '16', '18', '1', '8', '22', '23', '10', '25', '27', '24', '14', '42', '37', '44', '36', 'X', '59', '28', '56', '65', '32', '38', '9', '53', '33', '58', '61', '40', '64', '34', '57', '45', '48', '39', '43', '7', '19', '21', '66', '55', '46', '62', '20', '41', '26', '0', '11', '3', '51', '31', '17', '30', '4', '47', '50', '54', '60', '2', '6', '13', '15', '12'}


In [19]:
# verifying for correct period
def delinqu_mapper(row):
    if row['Monthly.Rpt.Prd'] == "03/01/2014":
        return True
    return False

In [28]:
performance_df_new['correct_month'] = performance_df_new.apply(delinqu_mapper, axis = 1)
performance_df_modfd = performance_df_new[performance_df_new["correct_month"]]

C:\Users\gaura\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
performance_df_modfd.head(10)

,LOAN_ID,Monthly.Rpt.Prd,Servicer.Name,LAST_RT,LAST_UPB,Loan.Age,Months.To.Legal.Mat,Adj.Month.To.Mat,Maturity.Date,MSA,...,NS_PROCS,CE_PROCS,RMW_PROCS,O_PROCS,NON_INT_UPB,PRIN_FORG_UPB_FHFA,REPCH_FLAG,PRIN_FORG_UPB_OTH,TRANSFER_FLG,correct_month
62,100013622306,03/01/2014,NaN,4.750,68259.31,50,130,129.0,01/2025,47300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,True
180,100022098429,03/01/2014,NaN,5.250,432083.23,49,311,311.0,02/2040,35620,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,True
383,100032715932,03/01/2014,NaN,5.375,89716.43,51,309,308.0,12/2039,43340,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,True
444,100033339771,03/01/2014,NaN,5.250,281467.05,48,312,312.0,03/2040,14260,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,True
498,100036162795,03/01/2014,NaN,4.625,4872.09,50,130,4.0,01/2025,47900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,True
575,100040783841,03/01/2014,NaN,4.750,239547.32,49,311,311.0,02/2040,39340,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,True
685,100044567305,03/01/2014,NaN,4.875,87435.60,49,311,284.0,02/2040,36780,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,True
763,100046088905,03/01/2014,NaN,4.750,259577.95,50,310,309.0,01/2040,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,True
831,100049953168,03/01/2014,NaN,5.250,272893.09,49,311,309.0,02/2040,35620,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,True
1012,100056508170,03/01/2014,NaN,5.000,132091.84,49,311,311.0,02/2040,47900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,True


In [30]:
delinquent  = performance_df_modfd.set_index("LOAN_ID")["Delq.Status"].to_dict()
acquisition_df["Delq.Status"] = acquisition_df["LOAN_ID"].map(delinquent)
del_df = acquisition_df["Delq.Status"].notna()
acquisition_df_modfd = acquisition_df[del_df]

In [31]:
acquisition_df_modfd.head(10)

,LOAN_ID,ORIG_CHN,Seller.Name,ORIG_RT,ORIG_AMT,ORIG_TRM,ORIG_DTE,FRST_DTE,OLTV,OCLTV,...,OCC_STAT,STATE,ZIP_3,MI_PCT,Product.Type,CSCORE_C,MI_TYPE,RELOCATION_FLG,FCC,Delq.Status
1,100013622306,R,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",4.750,87000,180,12/2009,02/2010,63,63.0,...,P,CA,932,NaN,FRM,785.0,NaN,N,False,0
3,100022098429,R,OTHER,5.250,461000,360,01/2010,03/2010,61,61.0,...,P,NY,112,NaN,FRM,NaN,NaN,N,False,0
8,100032715932,R,SUNTRUST MORTGAGE INC.,5.375,96000,360,12/2009,01/2010,75,75.0,...,I,LA,711,NaN,FRM,815.0,NaN,N,False,0
9,100033339771,R,OTHER,5.250,300000,360,02/2010,04/2010,74,74.0,...,P,ID,836,NaN,FRM,NaN,NaN,N,False,0
10,100036162795,R,"WELLS FARGO BANK, N.A.",4.625,200000,180,12/2009,02/2010,39,39.0,...,P,MD,217,NaN,FRM,NaN,NaN,N,False,0
12,100040783841,C,"CITIMORTGAGE, INC.",4.750,257000,360,01/2010,03/2010,52,52.0,...,P,UT,846,NaN,FRM,721.0,NaN,N,False,0
13,100044567305,C,OTHER,4.875,98000,360,01/2010,03/2010,54,54.0,...,S,WI,549,NaN,FRM,775.0,NaN,N,False,0
14,100046088905,C,"BANK OF AMERICA, N.A.",4.750,279000,360,12/2009,02/2010,64,64.0,...,P,CA,960,NaN,FRM,822.0,NaN,N,False,0
15,100049953168,B,"BANK OF AMERICA, N.A.",5.250,292000,360,01/2010,03/2010,80,80.0,...,P,NY,115,NaN,FRM,NaN,NaN,N,False,1
18,100056508170,R,"WELLS FARGO BANK, N.A.",5.000,141000,360,12/2009,03/2010,69,69.0,...,P,DC,200,NaN,FRM,NaN,NaN,N,False,0


In [36]:
fcc_dt = performance_df_new["FCC_DTE"].notna()
foreclosed = performance_df_new[fcc_dt]
foreclosed["FCC_DTE"]

8569      11/01/2012
13104     11/01/2016
19511     03/01/2015
24022     05/01/2016
29452     12/01/2013
31133     05/01/2013
36931     03/01/2016
39172     05/01/2011
61674     04/01/2016
76670     11/01/2013
86697     10/01/2015
99413     07/01/2018
111996    04/01/2017
141526    01/01/2016
178171    05/01/2017
193839    12/01/2013
210742    11/01/2012
215698    01/01/2011
222991    10/01/2016
237631    08/01/2016
239890    02/01/2013
240110    05/01/2018
245974    03/01/2016
272417    09/01/2015
275312    04/01/2015
284751    07/01/2017
340263    05/01/2014
344332    03/01/2012
347542    12/01/2015
354821    08/01/2015
             ...    
639971    07/01/2012
644007    05/01/2015
661099    08/01/2011
662713    08/01/2012
681489    05/01/2012
696433    08/01/2015
736462    06/01/2014
755059    02/01/2014
757356    02/01/2014
762627    12/01/2016
770477    12/01/2013
774090    07/01/2018
796217    04/01/2016
800179    01/01/2016
823638    05/01/2013
831755    04/01/2015
835779    04/

In [39]:
def day_mapper(row):
    diff = datetime.strptime(row["FCC_DTE"], '%m/%d/%Y') - datetime.strptime('03/01/2014', '%m/%d/%Y') 
    if diff.days >= 0 :
        return diff.days
    else:
        return np.NaN

In [40]:
foreclosed["Time_Delta"] = foreclosed.apply(lambda row: day_mapper(row), axis = 1)

C:\Users\gaura\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [41]:
foreclosed.head(10)

,LOAN_ID,Monthly.Rpt.Prd,Servicer.Name,LAST_RT,LAST_UPB,Loan.Age,Months.To.Legal.Mat,Adj.Month.To.Mat,Maturity.Date,MSA,...,CE_PROCS,RMW_PROCS,O_PROCS,NON_INT_UPB,PRIN_FORG_UPB_FHFA,REPCH_FLAG,PRIN_FORG_UPB_OTH,TRANSFER_FLG,correct_month,Time_Delta
8569,100446401716,11/01/2012,NaN,5.125,239556.43,34,326,0.0,01/2040,12580,...,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,False,NaN
13104,100683589832,11/01/2016,NaN,4.875,86107.60,80,280,0.0,03/2040,47260,...,NaN,NaN,2356.39,NaN,NaN,N,NaN,NaN,False,976.0
19511,100960628290,03/01/2015,NaN,5.000,93443.74,62,298,0.0,01/2040,12580,...,NaN,NaN,680.00,NaN,NaN,N,NaN,NaN,False,365.0
24022,101212627627,05/01/2016,NaN,5.000,317574.76,76,284,0.0,01/2040,41740,...,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,False,792.0
29452,101455126320,12/01/2013,NaN,5.250,110039.77,47,313,0.0,01/2040,49620,...,39935.16,NaN,NaN,NaN,NaN,N,NaN,NaN,False,NaN
31133,101558146692,05/01/2013,NaN,4.625,228098.09,40,140,0.0,01/2025,0,...,NaN,NaN,1409.95,NaN,NaN,N,NaN,NaN,False,NaN
36931,101865143451,03/01/2016,NaN,4.250,41220.56,72,108,0.0,03/2025,17140,...,NaN,NaN,4447.86,NaN,NaN,N,NaN,NaN,False,731.0
39172,101975325389,05/01/2011,NaN,5.940,60527.41,16,344,0.0,01/2040,19820,...,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,False,NaN
61674,103182946141,04/01/2016,NaN,4.875,60766.71,74,166,0.0,02/2030,28700,...,NaN,NaN,26.00,NaN,NaN,N,NaN,NaN,False,762.0
76670,103962879924,11/01/2013,NaN,5.375,294796.38,46,314,0.0,01/2040,0,...,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,False,NaN


In [43]:
delta  = foreclosed.set_index("LOAN_ID")["Time_Delta"].to_dict()
acquisition_df_modfd["Time_Delta"] = acquisition_df["LOAN_ID"].map(delta)

C:\Users\gaura\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [44]:
acquisition_df_modfd.head(10)

,LOAN_ID,ORIG_CHN,Seller.Name,ORIG_RT,ORIG_AMT,ORIG_TRM,ORIG_DTE,FRST_DTE,OLTV,OCLTV,...,STATE,ZIP_3,MI_PCT,Product.Type,CSCORE_C,MI_TYPE,RELOCATION_FLG,FCC,Delq.Status,Time_Delta
1,100013622306,R,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",4.750,87000,180,12/2009,02/2010,63,63.0,...,CA,932,NaN,FRM,785.0,NaN,N,False,0,NaN
3,100022098429,R,OTHER,5.250,461000,360,01/2010,03/2010,61,61.0,...,NY,112,NaN,FRM,NaN,NaN,N,False,0,NaN
8,100032715932,R,SUNTRUST MORTGAGE INC.,5.375,96000,360,12/2009,01/2010,75,75.0,...,LA,711,NaN,FRM,815.0,NaN,N,False,0,NaN
9,100033339771,R,OTHER,5.250,300000,360,02/2010,04/2010,74,74.0,...,ID,836,NaN,FRM,NaN,NaN,N,False,0,NaN
10,100036162795,R,"WELLS FARGO BANK, N.A.",4.625,200000,180,12/2009,02/2010,39,39.0,...,MD,217,NaN,FRM,NaN,NaN,N,False,0,NaN
12,100040783841,C,"CITIMORTGAGE, INC.",4.750,257000,360,01/2010,03/2010,52,52.0,...,UT,846,NaN,FRM,721.0,NaN,N,False,0,NaN
13,100044567305,C,OTHER,4.875,98000,360,01/2010,03/2010,54,54.0,...,WI,549,NaN,FRM,775.0,NaN,N,False,0,NaN
14,100046088905,C,"BANK OF AMERICA, N.A.",4.750,279000,360,12/2009,02/2010,64,64.0,...,CA,960,NaN,FRM,822.0,NaN,N,False,0,NaN
15,100049953168,B,"BANK OF AMERICA, N.A.",5.250,292000,360,01/2010,03/2010,80,80.0,...,NY,115,NaN,FRM,NaN,NaN,N,False,1,NaN
18,100056508170,R,"WELLS FARGO BANK, N.A.",5.000,141000,360,12/2009,03/2010,69,69.0,...,DC,200,NaN,FRM,NaN,NaN,N,False,0,NaN


In [48]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def delq_mapper(row):
    if is_number(row["Delq.Status"]) == True:
        return int(row["Delq.Status"])
    else:
        return 0 

In [50]:
acquisition_df_modfd["Delq.Status_Val"] = acquisition_df_modfd.apply(lambda row: delq_mapper(row), axis=1)

C:\Users\gaura\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [52]:
acquisition_df_new = acquisition_df_modfd[acquisition_df_modfd["Delq.Status_Val"] > 0]

In [54]:
acquisition_df_new.head(10)

,LOAN_ID,ORIG_CHN,Seller.Name,ORIG_RT,ORIG_AMT,ORIG_TRM,ORIG_DTE,FRST_DTE,OLTV,OCLTV,...,ZIP_3,MI_PCT,Product.Type,CSCORE_C,MI_TYPE,RELOCATION_FLG,FCC,Delq.Status,Time_Delta,Delq.Status_Val
15,100049953168,B,"BANK OF AMERICA, N.A.",5.250,292000,360,01/2010,03/2010,80,80.0,...,115,NaN,FRM,NaN,NaN,N,False,1,NaN,1
31,100091653134,R,OTHER,5.375,312000,360,12/2009,02/2010,80,80.0,...,265,NaN,FRM,NaN,NaN,N,False,1,NaN,1
110,100312066648,R,OTHER,5.250,160000,360,01/2010,03/2010,46,46.0,...,183,NaN,FRM,NaN,NaN,N,False,5,NaN,5
131,100393550601,C,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",4.750,80000,180,11/2009,01/2010,51,51.0,...,786,NaN,FRM,650.0,NaN,N,False,2,NaN,2
213,100657701189,C,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",4.875,220000,240,12/2009,02/2010,61,61.0,...,210,NaN,FRM,708.0,NaN,N,False,1,NaN,1
328,100929000903,R,OTHER,5.250,432000,360,11/2009,01/2010,66,66.0,...,109,NaN,FRM,NaN,NaN,N,False,20,NaN,20
341,100960628290,B,"CITIMORTGAGE, INC.",5.000,100000,360,12/2009,02/2010,63,63.0,...,212,NaN,FRM,NaN,NaN,N,True,1,365.0,1
587,101680663541,R,OTHER,4.875,77000,180,01/2010,03/2010,52,52.0,...,458,NaN,FRM,NaN,NaN,N,False,1,NaN,1
731,102038753630,R,OTHER,4.750,84000,180,01/2010,03/2010,80,80.0,...,89,NaN,FRM,NaN,NaN,N,False,3,NaN,3
762,102126520376,C,"CITIMORTGAGE, INC.",5.250,70000,180,10/2009,12/2009,47,47.0,...,708,NaN,FRM,624.0,NaN,N,False,1,NaN,1
